In [ ]:
!pip install ucimlrepo

In [35]:
from ucimlrepo import fetch_ucirepo

# List of dataset IDs and names (from UCI repository)
dataset_ids = {
    "wine_quality": 186,
    "concrete_strength": 165,
    "bike_sharing": 275,
}

# Dictionary to store datasets
datasets = {}

# Loop through dataset IDs and fetch data
for name, dataset_id in dataset_ids.items():
    dataset = fetch_ucirepo(id=dataset_id)

    # Store features (X) and target (y)
    datasets[name] = {
        "X": dataset.data.features,
        "y": dataset.data.targets,
        #"metadata": dataset.metadata,
        #"variables": dataset.variables
    }

In [36]:
datasets['wine_quality']

{'X':       fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
 0               7.4              0.70         0.00             1.9      0.076   
 1               7.8              0.88         0.00             2.6      0.098   
 2               7.8              0.76         0.04             2.3      0.092   
 3              11.2              0.28         0.56             1.9      0.075   
 4               7.4              0.70         0.00             1.9      0.076   
 ...             ...               ...          ...             ...        ...   
 6492            6.2              0.21         0.29             1.6      0.039   
 6493            6.6              0.32         0.36             8.0      0.047   
 6494            6.5              0.24         0.19             1.2      0.041   
 6495            5.5              0.29         0.30             1.1      0.022   
 6496            6.0              0.21         0.38             0.8      0.020   
 
       fr

In [47]:
from sklearn.datasets import fetch_openml

# List of OpenML dataset IDs (from your provided list)
openml_datasets = {
    "openml34538": 34538,
    "tst-census": 2000,
    "openml405": 405,
    "tmdb-movie-metadata": 4809,
    "openml503": 503,
    "openml558": 558,
    "openml308": 308
}

# Dictionary to store datasets
datasets2 = {}

# Loop through dataset IDs and fetch them
for name, dataset_id in openml_datasets.items():
    try:
        dataset = fetch_openml(data_id=dataset_id, as_frame=True, parser="pandas")

        # Store features (X) and target (y)
        datasets2[name] = {
            "X": dataset.data,
            "y": dataset.target,
            #"metadata": dataset.details
        }

        print(f"✅ Downloaded: {name} (ID {dataset_id})")

    except Exception as e:
        print(f"❌ Failed to download {name} (ID {dataset_id}): {e}")

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:1030: UserWarning: Version 12 of dataset QSAR-DATASET-FOR-DRUG-TARGET-CHEMBL1800 is inactive, meaning that issues have been found in the dataset. Try using a newer version from this URL: https://api.openml.org/data/v1/download/3532090/QSAR-DATASET-FOR-DRUG-TARGET-CHEMBL1800.arff
  warn(


❌ Failed to download openml34538 (ID 34538): Error tokenizing data. C error: Expected 36 fields in line 3, saw 38

❌ Failed to download tst-census (ID 2000): Dataset with data_id 2000 not found.
✅ Downloaded: openml405 (ID 405)
❌ Failed to download tmdb-movie-metadata (ID 4809): Dataset with data_id 4809 not found.
✅ Downloaded: openml503 (ID 503)
✅ Downloaded: openml558 (ID 558)
✅ Downloaded: openml308 (ID 308)


In [48]:
# Example usage: Accessing one available dataset
available_datasets = list(datasets.keys())  # Get valid dataset names

print("Available datasets:")
print(available_datasets)

Available datasets:
['wine_quality', 'concrete_strength', 'bike_sharing']


In [49]:
# Example usage: Accessing one available dataset
available_datasets2 = list(datasets2.keys())  # Get valid dataset names

print("Available datasets2:")
print(available_datasets2)

Available datasets2:
['openml405', 'openml503', 'openml558', 'openml308']


In [50]:
# prompt: merge dictionaries datasets and datasets2

# Merge the two dictionaries
merged_datasets = datasets.copy()  # Start with a copy of datasets
merged_datasets.update(datasets2)  # Add datasets2 to the copy



In [52]:
print(list(merged_datasets.keys()))

['wine_quality', 'concrete_strength', 'bike_sharing', 'openml405', 'openml503', 'openml558', 'openml308']


In [69]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.7 MB/s eta 0:00:00


In [72]:
import category_encoders as ce

regression_datasets = {}

# Example of accessing merged data
for k, v in merged_datasets.items():
  print(k)
  print(v['X'].values.shape)
  print(v['y'].values.ravel())
  encoder = ce.TargetEncoder(cols=v['X'])
  encoder.fit(v['X'], v['y'].values)
  X_cleaned = encoder.transform(v['X'])
  regression_datasets[k] = {"dfX": v['X'], "dfy":v['y'], "X":X_cleaned.values, "y":v['y'].values.ravel()}

wine_quality
(6497, 11)
[5 5 5 ... 6 7 6]
concrete_strength
(1030, 8)
[79.99 61.89 40.27 ... 23.7  32.77 32.4 ]
bike_sharing
(17379, 13)
[16 40 32 ... 90 61 49]
openml405
(4450, 202)
[0.       0.017173 0.035667 ... 0.377807 0.602378 0.200793]
openml503
(6574, 14)
[15.04 13.83 12.71 ... 29.58 28.79 22.08]
openml558
(8192, 32)
[0.049118 0.242579 0.       ... 0.006235 0.034258 0.06573 ]
openml308
(8192, 32)
[ 0.052627  0.001308  0.003834 ... -0.022139  0.00033  -0.000198]


In [73]:
# prompt: save `regression_datasets` in a downloadable format, using joblib

import joblib

# Assuming 'regression_datasets' is defined as in the previous code

# Specify the file path for saving
file_path = 'regression_datasets.joblib'

# Save the dictionary to a file
joblib.dump(regression_datasets, file_path, compress=3)  # compress for smaller file size

# To download the file in Google Colab:
from google.colab import files
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [74]:
from sklearn.linear_model import LinearRegression

for dataset in regression_datasets.values():
  model = LinearRegression()
  model.fit(dataset['X'], dataset['y'])
  print(model.score(dataset['X'], dataset['y']))

0.3937368603749972
0.7977039277771659
0.7068198451544414
0.9995364200580121
0.7159580456698464
0.9999622643211983
1.0
